# Setup

In [1]:
import json
import os
import sys
import warnings
from itertools import chain, combinations

sys.path.append('..')
sys.path.append('../scripts')

from tqdm import tqdm

import pandas as pd

from utils import read_csv_non_utf
from train_final_model import main as train_candidate_model

/Users/emiliolr/miniforge3/envs/life-hunting/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/Users/emiliolr/miniforge3/envs/life-hunting/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


## Utility functions

In [2]:
# Powerset function, ignoring the case of empty subset 
#  - https://docs.python.org/3/library/itertools.html#itertools-recipes  
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1))

# Getting combos of features to try

In [3]:
# Defining the number of models to compare for model selection
predictors = ['Body_Mass', 'Stunting_Pct', 'Literacy_Rate', 'Dist_Settlement_KM', 'Travel_Time_Large',
              'Protected_Area', 'Livestock_Biomass', 'Population_Density', 'Percent_Settlement_50km']
interactions = ['Body_Mass*Dist_Settlement_KM']
quadratics = ['I(Dist_Settlement_KM^2)', 'I(Population_Density^2)', 'I(Body_Mass^2)', 
              'I(Stunting_Pct^2)']

all_predictors = predictors + interactions + quadratics
all_combos = list(powerset(all_predictors))

In [4]:
# Thinning out possible combinations w/some reasonable rules:
#  1. Only include a quadratic if the original var is included
#  2. Only include interaction terms if both vars are included
#  3. Models must include body mass, population density, and at least one of 
#     the distance to settlement measures
final_combos = []

for c in all_combos:
    include = True
    
    #  rule 1
    for q in quadratics:
        if q in c:
            original_var = q.removeprefix('I(').removesuffix('^2)')
            
            if original_var not in c:
                include = False

    #  rule 2
    for i in interactions:
        if i in c:
            var1 = i.split('*')[0]
            var2 = i.split('*')[1]

            if (var1 not in c) or (var2 not in c):
                include = False

    #  rule 3
    if 'Body_Mass' not in c:
        include = False
    elif 'Population_Density' not in c:
        include = False
    elif ('Dist_Settlement_KM' not in c) and ('Percent_Settlement_50km' not in c):
        include = False

    #  including only if it satisfies all three rules
    if include:
        final_combos.append(c)

In [5]:
# Checking out the resulting set of models to test
print(f'After thinning out based on the ruleset, there are {len(final_combos)} models to try')
print(f'All models are unique: {len(set(final_combos)) == len(final_combos)}')

After thinning out based on the ruleset, there are 1728 models to try
All models are unique: True


In [6]:
# Constructing the formula based on the var subsets to try
nonzero_formula = []
zero_formula = []

for c in final_combos:
    formula = ' + '.join(c)
    nonzero_formula.append('RR ~ ' + formula + ' + (1|Country) + (1|Species) + (1|Study)')
    zero_formula.append('local_extirpation ~ ' + formula + ' + (1|Country) + (1|Species) + (1|Study)')

# Running the models + getting BIC

In [7]:
# A wrapper class to pass in params to the script like argparse
class ModelParams:
    def __init__(self, params):
        for k, v in params.items():
            setattr(self, k, v)

In [8]:
# Establishing generic model params
params = {'gdrive' : True,
          'dataset' : 'mammals_recreated', 
          'rebalance_dataset' : False,
          'tune_thresh' : False, 
          'outlier_cutoff' : 5, 
          'model_to_use' : 'pymer',
          'return_model' : True,
          'save_trained_model' : False,
          'verbose' : False,
          'pymer_reml' : False}

In [9]:
# Testing each candidate formula for the zero/nonzero models + recording the results...
formula = list(zip(zero_formula, nonzero_formula))
zero_results = {'formula' : [], 'aic' : [], 'bic' : [], 'log_likelihood' : []}
nonzero_results = {'formula' : [], 'aic' : [], 'bic' : [], 'log_likelihood' : []}

for z_form, nz_form in tqdm(formula):
    #  filling out the params w/the zero + nonzero formula to try
    params['pymer_zero_formula'] = z_form
    params['pymer_nonzero_formula'] = nz_form
    args = ModelParams(params)

    #  train the zero + nonzero models
    trained_model = train_candidate_model(args)

    #  record results in terms of AIC, BIC, and log-likelihood
    zero_results['formula'].append(z_form)
    zero_results['aic'].append(trained_model.zero_model.model.AIC)
    zero_results['bic'].append(trained_model.zero_model.model.BIC)
    zero_results['log_likelihood'].append(trained_model.zero_model.model.logLike)

    nonzero_results['formula'].append(nz_form)
    nonzero_results['aic'].append(trained_model.nonzero_model.model.AIC)
    nonzero_results['bic'].append(trained_model.nonzero_model.model.BIC)
    nonzero_results['log_likelihood'].append(trained_model.nonzero_model.model.logLike)

  2%|▉                                                         | 28/1728 [00:26<24:31,  1.16it/s]

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 



  2%|█▍                                                        | 43/1728 [00:40<24:45,  1.13it/s]

[1] "Model failed to converge with max|grad| = 0.11772 (tol = 0.002, component 1)"
[2] " \n"                                                                         

[1] "Model is nearly unidentifiable: very large eigenvalue\n - Rescale variables?"
[2] " \n"                                                                         



  3%|█▊                                                        | 53/1728 [00:50<25:27,  1.10it/s]

...

 99%|█████████████████████████████████████████████████████▌| 1712/1728 [1:00:51<00:46,  2.93s/it]

[1] "Model failed to converge with max|grad| = 0.111746 (tol = 0.002, component 1)"
[2] " \n"                                                                          

[1] "Model is nearly unidentifiable: very large eigenvalue\n - Rescale variables?"
[2] " \n"                                                                         



 99%|█████████████████████████████████████████████████████▋| 1719/1728 [1:01:15<00:27,  3.08s/it]

[1] "Model failed to converge with max|grad| = 0.117954 (tol = 0.002, component 1)"
[2] " \n"                                                                          

[1] "Model is nearly unidentifiable: very large eigenvalue\n - Rescale variables?"
[2] " \n"                                                                         



100%|██████████████████████████████████████████████████████| 1728/1728 [1:01:54<00:00,  2.15s/it]


In [10]:
# Turning results into dataframes + sorting by BIC
zero_results_df = pd.DataFrame.from_dict(zero_results)
nonzero_results_df = pd.DataFrame.from_dict(nonzero_results)

zero_results_df = zero_results_df.sort_values('bic', ascending = True)
nonzero_results_df = nonzero_results_df.sort_values('bic', ascending = True)

#  saving the results
save_dir = '../final_results/linear_model_feature_selection'
zero_results_df.to_csv(os.path.join(save_dir, 'zero_model_feature_selection.csv'))
nonzero_results_df.to_csv(os.path.join(save_dir, 'nonzero_model_feature_selection.csv'))

## Brief exploration of results

In [11]:
# Zero results
zero_results_df.head(5)

,formula,aic,bic,log_likelihood
50,local_extirpation ~ Body_Mass + Dist_Settlemen...,2100.659622,2155.283703,-1041.329811
154,local_extirpation ~ Body_Mass + Literacy_Rate ...,2094.879580,2155.573003,-1037.439790
191,local_extirpation ~ Body_Mass + Dist_Settlemen...,2095.131443,2155.824867,-1037.565722
199,local_extirpation ~ Body_Mass + Dist_Settlemen...,2095.403102,2156.096526,-1037.701551
412,local_extirpation ~ Body_Mass + Literacy_Rate ...,2090.101933,2156.864699,-1034.050966


In [26]:
# Best formula
zero_results_df.iloc[0]['formula']

'local_extirpation ~ Body_Mass + Dist_Settlement_KM + Travel_Time_Large + Livestock_Biomass + Population_Density + (1|Country) + (1|Species) + (1|Study)'

In [21]:
# Taking a look through the top 100 formula
print('\n\n'.join([f'{k}: {i}' for k, i in enumerate(zero_results_df.iloc[0 : 100]['formula'].to_list()) if '' in i]))

0: local_extirpation ~ Body_Mass + Dist_Settlement_KM + Travel_Time_Large + Livestock_Biomass + Population_Density + (1|Country) + (1|Species) + (1|Study)

1: local_extirpation ~ Body_Mass + Literacy_Rate + Dist_Settlement_KM + Travel_Time_Large + Livestock_Biomass + Population_Density + (1|Country) + (1|Species) + (1|Study)

2: local_extirpation ~ Body_Mass + Dist_Settlement_KM + Travel_Time_Large + Protected_Area + Livestock_Biomass + Population_Density + (1|Country) + (1|Species) + (1|Study)

3: local_extirpation ~ Body_Mass + Dist_Settlement_KM + Travel_Time_Large + Livestock_Biomass + Population_Density + I(Dist_Settlement_KM^2) + (1|Country) + (1|Species) + (1|Study)

4: local_extirpation ~ Body_Mass + Literacy_Rate + Dist_Settlement_KM + Travel_Time_Large + Protected_Area + Livestock_Biomass + Population_Density + (1|Country) + (1|Species) + (1|Study)

5: local_extirpation ~ Body_Mass + Literacy_Rate + Dist_Settlement_KM + Travel_Time_Large + Livestock_Biomass + Population_Densi

In [16]:
# Nonzero results
nonzero_results_df.head(5)

,formula,aic,bic,log_likelihood
36,RR ~ Body_Mass + Literacy_Rate + Dist_Settleme...,6997.489296,7056.816506,-3488.744648
44,RR ~ Body_Mass + Literacy_Rate + Travel_Time_L...,6998.575305,7057.902515,-3489.287653
6,RR ~ Body_Mass + Dist_Settlement_KM + Travel_T...,7004.701989,7058.096478,-3493.350994
2,RR ~ Body_Mass + Stunting_Pct + Dist_Settlemen...,7005.103869,7058.498358,-3493.551934
14,RR ~ Body_Mass + Travel_Time_Large + Populatio...,7005.920990,7059.315479,-3493.960495


In [32]:
# Best formula
nonzero_results_df.iloc[0]['formula']

'RR ~ Body_Mass + Literacy_Rate + Dist_Settlement_KM + Travel_Time_Large + Population_Density + (1|Country) + (1|Species) + (1|Study)'

In [33]:
# Taking a look through the top 100 formula
print('\n\n'.join([f'{k}: {i}' for k, i in enumerate(nonzero_results_df.iloc[0 : 100]['formula'].to_list()) if '' in i]))

0: RR ~ Body_Mass + Literacy_Rate + Dist_Settlement_KM + Travel_Time_Large + Population_Density + (1|Country) + (1|Species) + (1|Study)

1: RR ~ Body_Mass + Literacy_Rate + Travel_Time_Large + Population_Density + Percent_Settlement_50km + (1|Country) + (1|Species) + (1|Study)

2: RR ~ Body_Mass + Dist_Settlement_KM + Travel_Time_Large + Population_Density + (1|Country) + (1|Species) + (1|Study)

3: RR ~ Body_Mass + Stunting_Pct + Dist_Settlement_KM + Population_Density + (1|Country) + (1|Species) + (1|Study)

4: RR ~ Body_Mass + Travel_Time_Large + Population_Density + Percent_Settlement_50km + (1|Country) + (1|Species) + (1|Study)

5: RR ~ Body_Mass + Stunting_Pct + Dist_Settlement_KM + Travel_Time_Large + Population_Density + (1|Country) + (1|Species) + (1|Study)

6: RR ~ Body_Mass + Stunting_Pct + Dist_Settlement_KM + Population_Density + Percent_Settlement_50km + (1|Country) + (1|Species) + (1|Study)

7: RR ~ Body_Mass + Literacy_Rate + Dist_Settlement_KM + Population_Density + Pe